# Residual Momentum Strategy Analysis

This notebook implements and analyzes the Residual Momentum strategy for NYSE stocks.

In [ ]:
import sys

sys.path.append("../src")

import random
from datetime import datetime, timedelta

import pandas as pd

from data_collection import get_fama_french_factors, get_stock_data
from preprocessing import calculate_returns
from strategy import (
    calculate_adjusted_residuals,
    calculate_residuals,
    create_portfolio,
    fit_ff_models,
)
from visualization import plot_adjusted_residuals, plot_prices

## 1. Data Collection

In [ ]:
# Upload an Excel file containing stock data
stocks_df = pd.read_excel('../data/stocks.xlsx', skiprows=1)

# Delete the first column from the DataFrame
stocks_df = stocks_df.iloc[:, 1:]

# Display the first rows of the resulting DataFrame
stocks_df.head()

In [ ]:
# Define analysis period
end_date = datetime(2024, 3, 31)
start_date = end_date - timedelta(days=365*5)  # 5 years of data

# Get list of NYSE stocks
stocks_df = stocks_df[stocks_df["Exchange"] == "NYSE"]

# Get a list of symbols from the filtered stocks
nyse_stocks = stocks_df["Symbol"].str.replace('.K', '', regex=False).tolist()

# Download data
stock_data = get_stock_data(nyse_stocks, start_date, end_date)
ff_factors = get_fama_french_factors(start_date, end_date)

In [ ]:
# Calculate the proportion of stocks with missing data
missing_stocks = stock_data.isnull().sum() / len(stock_data)

# List the stocks that will not be considered for the analysis
missing_stocks = missing_stocks[missing_stocks == 1].index

# Define the list of stocks with available data for analysis
available_stocks = list(set(nyse_stocks) - set(missing_stocks))

In [ ]:
# List of stock symbols
some_stocks = random.sample(available_stocks, 10)

plot_prices(some_stocks, stock_data)

## 2. Data Preprocessing

In [10]:
returns_df = calculate_returns(stock_data)

## 3. Strategy Implementation

In [11]:
ff_models = fit_ff_models(returns_df, ff_factors, available_stocks)
residual_df = calculate_residuals(returns_df, ff_models, ff_factors, available_stocks)

In [13]:
risk_adjusted_residuals = calculate_adjusted_residuals(residual_df)
buy_assets, sell_assets = create_portfolio(risk_adjusted_residuals, top_pct=10, bottom_pct=10)
risk_adjusted_residuals.to_csv('../results/portfolio_performance.csv')

### 3.1 Portfolio

In [ ]:
buy_assets

In [ ]:
sell_assets

## 4. Results Analysis

In [ ]:
plot_adjusted_residuals(risk_adjusted_residuals)

## 5. Conclusions

### Recommendations

Having applied the Residual Momentum method to the available data of stocks from the New York Stock Exchange, it is suggested that stocks with higher residuals could be considered as buy candidates, as the model indicates these stocks fit better and may involve lower risk. On the other hand, stocks with lower residuals could be considered sell candidates, since the model suggests higher risk and that predictions may deviate further from actual values.

### Stocks with Highest Risk-Adjusted Residuals (Buy Candidates)

|  Stock  |    Risk-Adjusted Residual   |
|---------|-----------------------------|
|NVR      | 6.764213                    |
|TRV      | 7.042893                    |
|TM       | 7.539809                    |
|CB       | 7.595617                    |
|MUFG     | 7.903012                    |
|AIG      | 8.021970                    |
|ECL      | 8.083168                    |
|ITUB     | 8.282844                    |
|NOW      | 8.350461                    |
|HMC      | 8.402053                    |
|SMFG     | 8.835399                    |
|DLR      | 9.078423                    |
|PH       | 9.389937                    |
|DELL     | 10.276290                   |
|IBM      | 11.109896                   |
|GE       | 11.745092                   |
|UBER     | 12.721726                   |
|WFC      | 12.728989                   |
|MAC      | 12.949400                   |
|MFG      | 13.019338                   |
|MLM      | 13.536678                   |
|PNR      | 13.629176                   |
|LLY      | 13.706111                   |
|HLT      | 14.804893                   |
|TDG      | 20.630775                   |
|JPM      | 29.856767                   |

### Stocks with Lowest Risk-Adjusted Residuals (Sell Candidates)

|  Stock  |    Risk-Adjusted Residual   |
|---------|-----------------------------|
| PFE     | -44.833280                  |
| AMX     | -35.110769                  |
| KO      | -28.384738                  |
| STT     | -22.632032                  |
| PM      | -19.269756                  |
| MAA     | -17.624162                  |
| ETR     | -15.904199                  |
| ENB     | -14.801951                  |
| UPS     | -13.890470                  |
| EIX     | -13.580058                  |
| MCD     | -12.531976                  |
| BMY     | -11.902264                  |
| LPL     | -11.729517                  |
| ED      | -11.373881                  |
| MO      | -11.365633                  |
| AWK     | -11.092975                  |
| KMB     | -10.290196                  |
| BBY     | -10.104498                  |
| ABEV    |  -9.971825                  |
| CPB     |  -9.695247                  |
| MGM     |  -9.369015                  |
| BNS     |  -8.979885                  |
| NKE     |  -7.966800                  |
| JNJ     |  -7.892484                  |
| GGB     |  -7.290168                  |
| HDB     |  -7.116820                  |